### Load Functions

In [1]:
from simple_trade import download_data, compute_indicator
from simple_trade import BandTradeBacktester
from simple_trade import Optimizer

### Load Data

In [2]:
ticker = 'SPY'
start_date = '2020-01-01'
end_date = '2024-12-31'

data = download_data(ticker, start_date, end_date)

### Load Optimization Parameters

In [3]:
# --- Optimization Parameters ---
# Define the parameter grid to search
param_grid = {
    'rsi_window': [12, 14, 16],
    'rsi_lower_threshold': [20, 30],
    'rsi_upper_threshold': [70, 80],
}

# Define constant parameters for the backtester
initial_capital = 100000
commission_fee_long = 0.001 # 0.1%
commission_fee_short = 0.001 # 0.1%
constant_params = {
    'initial_cash': initial_capital, 
    'commission_long': commission_fee_long,
    'commission_short': commission_fee_short,
    'price_col': 'Close'
}

# Define the metric to optimize and whether to maximize or minimize
metric_to_optimize = 'total_return_pct'
maximize_metric = True

### Define Wrapper Function

In [4]:
def run_band_trade_with_windows(data, rsi_upper_threshold, rsi_lower_threshold, rsi_window, **kwargs):
    """
    Wrapper function to compute indicators and run the band trade backtest.
    """
    # Work on a copy of the data
    df = data.copy()
    
    # Set up threshold columns
    upper_threshold_col = 'RSI_Upper'
    lower_threshold_col = 'RSI_Lower'
    df[upper_threshold_col] = rsi_upper_threshold
    df[lower_threshold_col] = rsi_lower_threshold

    # Compute RSI indicator
    df = compute_indicator(df, indicator='rsi', parameters={'window': rsi_window}, columns={'close_col': 'Close'})
    indicator_col = f'RSI_{rsi_window}'
    
    # Create a backtester instance
    backtester = BandTradeBacktester(
        initial_cash=kwargs.pop('initial_cash', 10000),
        commission_long=kwargs.pop('commission_long', 0.001),
        commission_short=kwargs.pop('commission_short', 0.001),
        short_borrow_fee_inc_rate=kwargs.pop('short_borrow_fee_inc_rate', 0.0),
        long_borrow_fee_inc_rate=kwargs.pop('long_borrow_fee_inc_rate', 0.0)
    )
    
    # Run the backtest with the properly named method
    return backtester.run_band_trade(
        data=df,
        indicator_col=indicator_col,
        upper_band_col=upper_threshold_col,
        lower_band_col=lower_threshold_col,
        **kwargs
    )

### Start Optimizer

In [5]:
# --- Instantiate and Run Optimizer ---
print("Initializing Optimizer...")
optimizer = Optimizer(
    data=data,
    backtest_func=run_band_trade_with_windows,  # Use our wrapper function
    param_grid=param_grid,
    metric_to_optimize=metric_to_optimize,
    maximize_metric=maximize_metric,
    constant_params=constant_params
)

print("\nRunning Optimization (Parallel)...")
# Run optimization with parallel processing (adjust n_jobs as needed)
results = optimizer.optimize(parallel=True, n_jobs=1) # n_jobs=-1 uses all available cores

# --- Display Results ---
print("\n--- Optimization Results ---")

# Unpack results
best_params, best_metric_value, all_results = results

print("\n--- Top 5 Parameter Combinations ---")
# Sort results for display
sorted_results = sorted(all_results, key=lambda x: x[1], reverse=maximize_metric)
for i, (params, metric_val) in enumerate(sorted_results[:5]):
    print(f"{i+1}. Params: {params}, Metric: {metric_val:.4f}")

Initializing Optimizer...
Generated 12 parameter combinations.

Running Optimization (Parallel)...
Starting optimization for 12 combinations...
Metric: total_return_pct (Maximize) | Parallel: True (n_jobs=1)
Using 1 parallel jobs.
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Computing RSI...
Optimization finished in 1.59 seconds.
Best Parameters found: {'rsi_window': 12, 'rsi_lower_threshold': 30, 'rsi_upper_threshold': 80}
Best Metric Value (total_return_pct): 38.4200

--- Optimization Results ---

--- Top 5 Parameter Combinations ---
1. Params: {'rsi_window': 12, 'rsi_lower_threshold': 30, 'rsi_upper_threshold': 80}, Metric: 38.4200
2. Params: {'rsi_window': 16, 'rsi_lower_threshold': 30, 'rsi_upper_threshold': 80}, Metric: 28.1200
3. Params: {'rsi_window': 14, 'rsi_lower_threshold': 30, 'rsi_upper_threshold': 80}, Metric: 27.9500
4. Params: {'

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.5s finished
